# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'
sns.set()
import shap


from metrics import compute_score, just_mAP_mRCE
from import_data import import_data, categorical_features, timestamp_features, targets
from cross_validate import cross_validate, cv_metrics

In [3]:
%%time
df = import_data('../data/raw/sample_200k_rows')
df

CPU times: user 1.99 s, sys: 90.1 ms, total: 2.08 s
Wall time: 2.1 s


,tweet_id,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply,retweet,retweet_with_comment,like
tweet_timestamp,,,,,,,,,,,,,,,,,,,
2021-02-04 00:00:00,3A582115842794F64DAD3AF86C234193,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-04 00:00:00,71E966651BCB91EE3BA550ACBB896657,130569,2447,True,2008-01-04 18:41:54,C301F5A3440C055079A799F39024F7C4,184,255,False,2011-11-11 11:48:41,False,False,False,False,False
2021-02-04 00:00:02,E0FD49F10FA76EE26A357B730D7A9597,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-04 00:00:02,19E8DF58CC1CDA53FF8EF7DBF5F7AFA4,361813,273,True,2011-08-26 00:44:12,55BDB6B97CC58181915D41605E4DA6C5,258,497,False,2015-04-11 11:10:50,False,False,False,False,True
2021-02-04 00:00:03,028199B091010E869A5FADDF296CD48D,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-04 00:00:03,40F3B120A620450028E74DA69A74580A,581,576,False,2020-10-05 07:26:18,FD07DE4F172A889887D5B409F2512399,1368,1989,False,2020-04-10 07:47:30,True,False,False,False,False
2021-02-04 00:00:06,9DD68C8614B87009EF39AB18FDB46219,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-04 00:00:06,818C1DE428D290E133A26BF8E5CD03C6,7600462,335,True,2009-03-05 21:10:42,60D1B6996C040E67CB3F15E5E14BA0E1,951,725,False,2013-11-26 01:39:07,False,False,False,False,True
2021-02-04 00:00:09,4D68518C190D969C03E315224F349358,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-04 00:00:09,33CA178513C1EDAC5716350A612569A6,20387,7586,True,2009-06-10 22:15:47,3B5E75CC0E9C08757B8E2BBFC26B02D2,48,423,False,2010-09-05 07:02:43,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 23:59:21,F0FC42D8DFB304DAED14ECFA96888637,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:59:21,2369E79F89DBFDD7D6D7629958D2269D,1102,3887,False,2020-09-17 18:27:15,C21CDDB041C38B02AC0DC9D0FB1A9151,178,209,False,2015-09-26 13:06:12,True,False,False,False,False
2021-02-24 23:59:25,C285954F3C93513A3356253B60827313,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:59:25,65EA57BA5FF089ECE5CA2BA9FF3CA957,407236,1895,True,2008-03-12 17:43:25,EEDFE24D68A7DE5D6F94170A3FFA1917,8414,5093,False,2015-01-11 15:01:47,False,False,False,False,False
2021-02-24 23:59:35,8C10725EE116DB3070246773965C99BB,TopLevel,B8B04128918BBF54E2E178BFF1ABA833,2021-02-24 23:59:35,907082A1742C7248FD887AFDD079714A,509,557,False,2019-01-24 00:41:24,164AC1EC679B86A262FA831393668AB5,426,1206,False,2009-06-10 18:54:44,False,False,False,False,False


# EDA

# Feature Engineering

In [4]:
class Transformer:
    enabled_features = [
        # Tweet features
        "tweet_type",
        "language",
        "tweet_timestamp",
        # Engaged-with User (i.e., Engagee) Features
        "engaged_with_user_follower_count",
        "engaged_with_user_following_count",
        "engaged_with_user_is_verified",
        "engaged_with_user_account_creation",
        # Engaging User (i.e., Engager) Features
        "engaging_user_follower_count",
        "engaging_user_following_count",
        "engaging_user_is_verified",
        "engaging_user_account_creation",
        # Engagement features
        "engagee_follows_engager"
    ]
    
    def __init__(self):
        pass
    
    def fit_transform(self, train_df):
        self._categorical_and_timestamps_to_int(train_df)
        return train_df[self.enabled_features]

    def transform(self, test_df):
        self._categorical_and_timestamps_to_int(test_df)
        return test_df[self.enabled_features]
    
    def _categorical_and_timestamps_to_int(self, df):
        for f in categorical_features:
            df[f] = df[f].cat.codes
        for f in timestamp_features:
            df[f] = pd.to_numeric(df[f])

In [5]:
feature_engineer = Transformer()
cv_models, cv_results = cross_validate(df, feature_engineer)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

+0.2097 ± 0.0035
-4.5889 ± 0.6478


In [9]:
for metric_lst in cv_metrics(cv_results):
    metric_lst = np.array(metric_lst)
    print(f"{metric_lst.mean():+.4f} ± {metric_lst.std():.4f} (± 1σ)")

+0.2097 ± 0.0035 (± 1σ)
-4.5889 ± 0.6478 (± 1σ)


In [6]:
# Feature Selection


# Feature Engineering
## Target Encoding of categorical features

# target_encoders = {}
# for feature in categorical_features:
#     for target in targets:
#         te = ce.TargetEncoder(smoothing=20.0)
#         te.fit(df_train[feature], df_train[target])
#         target_encoders[f'TE_{feature}__{target}'] = te

# for tmp_df in [df_train_features, df_valid_features, df_test_features]:
#     for name, te in target_encoders.items():
#         tmp_df[name] = te.transform(tmp_df[te.cols[0]])[te.cols[0]]

        
# for col in categorical_features:
#     df_train_features = df_train_features.drop(col, axis=1)
#     df_valid_features = df_valid_features.drop(col, axis=1)
#     df_test_features = df_test_features.drop(col, axis=1)

# Column size assertions
# assert df_train_features.shape[1] == df_valid_features.shape[1] == df_test_features.shape[1], \
#     "train, valid, test should have the same number of columns"

# Model Training and Evaluation

What the current top leaderboard submission looks like:
```
mAP:                        0.3115
mRCE:                      16.4698
```
Leaderboard: https://recsys-twitter.com/leaderboard/all-submissions

# Model analysis

In [7]:
# for target, model in models.items():
#     plt.title(target)
#     shap.plots.beeswarm(shap.Explainer(model)(df_test_features))